<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [6]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2025-09-22 18:22:25--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.43.25, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2025-09-22 18:22:26--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1.1’

SO_vectors_200.bin?   0%[                    ]   3.02M  1.45MB/s               ^C


In [ ]:
!pip3 install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
    --------------------------------------- 0.5/24.0 MB 3.4 MB/s eta 0:00:07
   -- ------------------------------------- 1.3/24.0 MB 5.2 MB/s eta 0:00:05
   ----- ---------------------------------- 3.1/24.0 MB 5.8 MB/s eta 0:00:04
   ----- ---------------------------------- 3.4/24.0 MB 5.9 MB/s eta 0:00:04
   --------- ------------------------------ 5.5/24.0 MB 6.0 MB/s eta 0:00:04
   ----------- ---------------------------- 6.8/24.0 MB 5.9 MB/s eta 0:00:03
   ------------- -------------------------- 8.1/24.0 MB 5.9 MB/s eta 0:00:03
   --------------- ------------------------ 9.2/24.0 MB 5.8 MB/s eta 0:00:03
   ---------------- ----------------------- 10.0/24.0 MB 5.6 MB/s eta 0:00:03
   ------------------ --------------------- 11.0/24.0 MB 5.5 MB/s eta 0:00:03
   ------------------- -------------------- 11.8/24.0 MB 5.4 MB/s eta 0:00:03
   --------------------- ------------------ 12.8/24.0 MB 5.3 MB/s eta 0:00:03
  


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format(r"C:\Project\DeepLearningSchool\NLP\SO_vectors_200.bindownload=1", binary=True)

In [2]:
len(wv_embeddings[0])

200

In [3]:
len(wv_embeddings)

1787145

In [4]:
print(list(wv_embeddings.most_similar(100)))

[('problem', 0.8884832262992859), ('issues', 0.840576708316803), ('problems', 0.7842557430267334), ('bug', 0.6874933838844299), ('isssue', 0.6070330739021301), ('issue-', 0.606260359287262), ('woes', 0.60285884141922), ('issuse', 0.5781072378158569), ('isse', 0.5734156370162964), ('weirdness', 0.5670750737190247)]


In [5]:
print(list(wv_embeddings.key_to_index.keys())[:50])

['use', 'code', 'using', 'like', 'will', 'want', 'need', 'get', 'file', 'one', 'just', 'data', 'way', '1', 'also', 'function', 'problem', 'error', '-', 'example', '2', 'class', 'work', 'value', 'method', 'first', 'set', 'see', 'following', '0', 'user', 'make', 'new', 'add', 'know', 'try', 'time', 'something', 'http', 'trying', 'create', 'object', 'now', 'page', 'server', 'string', 'help', 'app', 'list', 'table']


#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [6]:
word = 'cat'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [7]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [8]:
wv_embeddings.most_similar("dog")

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398715019226),
 ('bird', 0.7315488457679749),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349342346191),
 ('labrador', 0.7209131717681885)]

***Ваш ответ:*** 'cat' не входит в топ-5 ближайших слов к 'dog'. Но слово `cats` входит (4 место)

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [9]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()
tokenizer.tokenize("Hello, my 1.friend")

['Hello', 'my', '1', 'friend']

In [10]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    vec_words = []

    for word in tokenizer.tokenize(question):
        if word in embeddings:
            vec_words.append(embeddings[word])

    return np.mean(vec_words, axis=0) if vec_words else np.zeros(dim)

Проверка

In [20]:
v1 = question_to_vec("cats wolf", wv_embeddings, tokenizer)

In [21]:
v2 = question_to_vec("a", wv_embeddings, tokenizer)

In [22]:
v1.shape == v2.shape

True

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [11]:
# Предложение
question = "I love neural networks"

np.round_(question_to_vec(question, wv_embeddings, tokenizer)[2], 2)


-1.29

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $$\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0 \\
   0, &x \qeq 0
 \end{cases}
\end{equation*}$$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)


In [12]:
import math


def dcg(rank_q_i, k):
    if rank_q_i <= k:
        return 1/(math.log2(1 + rank_q_i))
    return 0

dcg(9, 10)

0.3010299956639812

**Ответ: 0,3**


Можно заметить, что если rank_q_i < k_1, то повышение k_1 не привдет к изменению.

#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?

Надо, чтобы Hits@47 -> max
И, чтобы DCG@1 -> min

Hits@47 стремится к максимум когда N -> min, а rank_q <= 47. Это от 1 до 47.

DCG@1 стремится к минимуму, когда rank_q > k. Это 0.

Получается максимум при rank_q равным от 2 до 47. Ответ 1

Рассмотри для произвольного N > 0:

Для DSG@1 важно, чтобы любое rank_q > k. Тогда все равно 0.

Для Hits@47 -> будет к максимален, когда каждый rank_q Будет в это диапозоне от 1 до 47. Причем, заполнение должно идти. Анологично максимум равен 1. так как всего $\frac{1}{N} * \sum_{i=1}^{N}[rankq_i <= k] = \frac{1}{N} * N$. Если все rank_q в пределах от 1 до 47. Но нам нельзя занимать 1 место иначе, тогда DCG@1 станет не нулевым. Тогда для этого случая получаем следующее.

ОТВЕТ:

Если $ N \in [1; 46]$, то максимум равен 1.

Если $ N \in [47; +\inf) $, то максимум $ \frac{1}{N} * (N - 1)$


Скорее всего предлагается N = 1, то  ответ 1






### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [13]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть Hits@k
    """
    # Подсчитываем количество дубликатов, чей ранг <= k
    '''your code'''
    s = 0
    for i in dup_ranks:
        if i <= k:
            s+=1

    return s/len(dup_ranks)

In [14]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0.0
Hits@4 = 1.0


In [15]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть DCG@k
    """
    s = 0
    for i in dup_ranks:
        if i <= k:
            s += 1/math.log2((i+1))

    return s/len(dup_ranks)

In [16]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@10 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@10 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [29]:
!pip3 install pandas

   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.3 MB 932.9 kB/s eta 0:00:12
   -- ------------------------------------- 0.8/11.3 MB 1.0 MB/s eta 0:00:11
   ---- ----------------------------------- 1.3/11.3 MB 1.2 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/11.3 MB 1.3 MB/s eta 0:00:08
   ------- -------------------------------- 2.1/11.3 MB 1.4 MB/s eta 0:00:07
   -------- ------------------------------- 2.4/11.3 MB 1.5 MB/s eta 0:00:07
   ---------- ----------------------------- 2.9/11.3 MB 1.6 MB/s eta 0:00:06
   ------------ --------------------------- 3.4/11.3 MB 1.7 MB/s eta 0:00:05
   -------------- ------------------------- 4.2/11.3 MB 1.9 MB/s eta 0:00:04
   ----------------- ----


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd

In [18]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [19]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip "/home/devoloper/projects/DeepLearningSchool/NLP/some_data/stackoverflow_similar_questions.zip"

Archive:  /home/devoloper/projects/DeepLearningSchool/NLP/some_data/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [20]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [21]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [22]:
len(validation_data)

3760

Размер нескольких первых строк

In [23]:
for i in range(25):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001
11 1001
12 1001
13 1001
14 1001
15 1001
16 1001
17 1001
18 1001
19 1001
20 1001
21 1001
22 1001
23 1001
24 1001
25 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [37]:
!pip3 install scikit-learn

   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.9 MB 5.0 MB/s eta 0:00:02
   ------- -------------------------------- 1.6/8.9 MB 5.2 MB/s eta 0:00:02
   -------------- ------------------------- 3.1/8.9 MB 5.3 MB/s eta 0:00:02
   ----------------------- ---------------- 5.2/8.9 MB 6.2 MB/s eta 0:00:01
   ------------------------------- -------- 7.1/8.9 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 8.9/8.9 MB 7.1 MB/s eta 0:00:00

   ------------- -------------------------- 1/3 [joblib]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ------------- 2/3 [scikit-learn]
   -------------------------- ---


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [25]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_vector = question_to_vec(question, embeddings, tokenizer, dim).reshape(1, -1)

    ranked_list = []
    for i, candidate in enumerate(candidates):
        candidate_vector = question_to_vec(candidate, embeddings, tokenizer, dim).reshape(1, -1)

        similarity = cosine_similarity(question_vector, candidate_vector)[0][0]
        ranked_list.append((similarity, i, candidate))

    # Сортируем по убыванию схожести
    ranked_list.sort(key=lambda x: x[0], reverse=True)

    return [(item[1], item[2]) for item in ranked_list]


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [26]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [27]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [28]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results'), #скрыт
            (1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

021 для эксперемента 2

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [43]:
!pip3 install tqdm


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
!pip3 install ipywidgets

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.2 MB 1.3 MB/s eta 0:00:02
   -------------- ------------------------- 0.8/2.2 MB 1.4 MB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.2 MB 1.5 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.2 MB 1.6 MB/s eta 0:00:01
   --------------------------------- ------ 1.8/2.2 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 1.7 MB/s eta 0:00:00

   -------------------------- ------------- 2/3 [ipywidgets]
   ---------------------------------------- 3/3 [ipywidgets]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
from tqdm import tqdm

In [30]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

 27%|██▋       | 1000/3760 [03:28<09:34,  4.81it/s]


In [31]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<00:00, 4464.40it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [47]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [33]:
train_data[5566]

['Why prefer start + (end - start) / 2 over (start + end) / 2 when calculating the middle of an array?',
 'Why are binary search index computed this way?']

In [48]:
# Создаем общий корпус текстов
corpus = []

#  TODO: ВОЗМОЖНО непраильно, перезапустить !!!!!!!!!!

'''your code'''
for texts in train_data:
    combined_text = texts[0] + ' ' + texts[1]
    tokens = tokenizer.tokenize(combined_text.lower())
    corpus.append(tokens)

print(f"Пример первой записи в корпусе: {corpus[0]}")

Пример первой записи в корпусе: ['con', 'onv', 'nve', 'ver', 'ert', 'rti', 'tin', 'ing', 'ng ', 'g s', ' st', 'str', 'tri', 'rin', 'ing', 'ng ', 'g t', ' to', 'to ', 'o l', ' li', 'lis', 'ist', 'st ', 't c', ' co', 'con', 'onv', 'nve', 'ver', 'ert', 'rt ', 't g', ' go', 'goo', 'oog', 'ogl', 'gle', 'le ', 'e r', ' re', 'res', 'esu', 'sul', 'ult', 'lts', 'ts ', 's o', ' ob', 'obj', 'bje', 'jec', 'ect', 'ct ', 't (', ' (p', '(pu', 'pur', 'ure', 're ', 'e j', ' js', 'js)', 's) ', ') t', ' to', 'to ', 'o p', ' py', 'pyt', 'yth', 'tho', 'hon', 'on ', 'n o', ' ob', 'obj', 'bje', 'jec', 'ect']


In [41]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=8,                # Размер окна контекста
    min_count=1,             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [42]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

 27%|██▋       | 1000/3760 [03:24<09:23,  4.89it/s]


In [43]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<00:00, 1907.51it/s]

DCG@   1: 0.202 | Hits@   1: 0.202
DCG@   5: 0.253 | Hits@   5: 0.298
DCG@  10: 0.268 | Hits@  10: 0.347
DCG@ 100: 0.317 | Hits@ 100: 0.589
DCG@ 500: 0.347 | Hits@ 500: 0.827
DCG@1000: 0.365 | Hits@1000: 1.000


Другая токенизация из модуля не словами, а n граммами (3):

In [46]:
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text, n=3):
        text = text.lower()

        ngrams = []
        for i in range(len(text) - n + 1):
            ngrams.append(text[i:i+n])

        return ngrams
tokenizer = MyTokenizer()
tokenizer.tokenize("Hello, my 1.friend")

['hel',
 'ell',
 'llo',
 'lo,',
 'o, ',
 ', m',
 ' my',
 'my ',
 'y 1',
 ' 1.',
 '1.f',
 '.fr',
 'fri',
 'rie',
 'ien',
 'end']

In [51]:
# Создаем общий корпус текстов
corpus = []

#  TODO: ВОЗМОЖНО непраильно, перезапустить !!!!!!!!!!

'''your code'''
for texts in train_data:
    combined_text = texts[0] + ' ' + texts[1]
    tokens = tokenizer.tokenize(combined_text.lower())
    corpus.append(tokens)

print(f"Пример первой записи в корпусе: {corpus[0]}")

Пример первой записи в корпусе: ['con', 'onv', 'nve', 'ver', 'ert', 'rti', 'tin', 'ing', 'ng ', 'g s', ' st', 'str', 'tri', 'rin', 'ing', 'ng ', 'g t', ' to', 'to ', 'o l', ' li', 'lis', 'ist', 'st ', 't c', ' co', 'con', 'onv', 'nve', 'ver', 'ert', 'rt ', 't g', ' go', 'goo', 'oog', 'ogl', 'gle', 'le ', 'e r', ' re', 'res', 'esu', 'sul', 'ult', 'lts', 'ts ', 's o', ' ob', 'obj', 'bje', 'jec', 'ect', 'ct ', 't (', ' (p', '(pu', 'pur', 'ure', 're ', 'e j', ' js', 'js)', 's) ', ') t', ' to', 'to ', 'o p', ' py', 'pyt', 'yth', 'tho', 'hon', 'on ', 'n o', ' ob', 'obj', 'bje', 'jec', 'ect']


In [49]:
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=8,                # Размер окна контекста
    min_count=1,             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [50]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<?, ?it/s]

DCG@   1: 0.286 | Hits@   1: 0.286
DCG@   5: 0.339 | Hits@   5: 0.386
DCG@  10: 0.355 | Hits@  10: 0.438
DCG@ 100: 0.394 | Hits@ 100: 0.638
DCG@ 500: 0.424 | Hits@ 500: 0.867
DCG@1000: 0.438 | Hits@1000: 1.000


разделение на n-граммы увеличело результаты, но не на слишком колосальные результаты. 0.08. Причем исользовались n-граммы просто разделяюшие весь текст по 3 символа (с пробелами).

In [67]:
!pip3 install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   --------------------------- ------------ 1.0/1.5 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.3 MB/s eta 0:00:00

   ------------- -------------------------- 1/3 [click]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   -------------------------- ------------- 2/3 [nltk]
   ---------------------------------------- 3/3 [nltk]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [68]:
from nltk import ngrams

In [97]:
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text: str, n=3):
        text = re.sub(r'[^\w\s]', '', text).lower()

        s = []
        # 2. Разбиение текста на слова и итерация по ним
        for word in text.split():
            # 3. Генерация символьных n-грамм для каждого слова
            if len(word) >= n:
                for i in range(len(word) - n + 1):
                    s.append(word[i:i+n])
            else:
                s.append(word)

        return s
tokenizer = MyTokenizer()
tokenizer.tokenize("Hello, my 1. friend")

['hel', 'ell', 'llo', 'my', '1', 'fri', 'rie', 'ien', 'end']

In [ ]:
# Создаем общий корпус текстов
corpus = []


'''your code'''
for texts in train_data:
    combined_text = texts[0] + ' ' + texts[1]
    tokens = tokenizer.tokenize(combined_text.lower())
    corpus.append(tokens)

print(f"Пример первой записи в корпусе: {corpus[0]}")

Пример первой записи в корпусе: ['con', 'onv', 'nve', 'ver', 'ert', 'rti', 'tin', 'ing', 'str', 'tri', 'rin', 'ing', 'to', 'lis', 'ist', 'con', 'onv', 'nve', 'ver', 'ert', 'goo', 'oog', 'ogl', 'gle', 'res', 'esu', 'sul', 'ult', 'lts', 'obj', 'bje', 'jec', 'ect', 'pur', 'ure', 'js', 'to', 'pyt', 'yth', 'tho', 'hon', 'obj', 'bje', 'jec', 'ect']


In [99]:
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=8,                # Размер окна контекста
    min_count=1,             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [100]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<00:00, 6007.60it/s]

DCG@   1: 0.377 | Hits@   1: 0.377
DCG@   5: 0.459 | Hits@   5: 0.533
DCG@  10: 0.474 | Hits@  10: 0.578
DCG@ 100: 0.515 | Hits@ 100: 0.781
DCG@ 500: 0.534 | Hits@ 500: 0.926
DCG@1000: 0.542 | Hits@1000: 1.000


Результаты еще увеличились при дополнительном предпроцессинге текста.

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?

N-грамм, потому что мы кодируем не отдельные слова, а набор грамм, которые лучше работают.

* Помогает ли нормализация слов?

Да помогает, так как очищает данные от лишнего.

* Какие эмбеддинги лучше справляются с задачей и почему?

Лучше уже заготовленные, но если обучить Word2Vec лучше, то возможно он будет лучше, под определенный набор данных.

* Почему получилось плохое качество решения задачи?

Токинезация слов, 

* Предложите свой подход к решению задачи.


Лучше токинизировать слова на n-граммы (уже преведенно в экспирименте, значительный прирост в качестве), 

## Вывод:

Качество модели зависит от качества данных



